<h2>matplotlibによるjpegファイル読み込みのメモ</h2>
<br>
<h3>①jpegファイルの表示</h3>

<p>まずドキュメントの<a href= "https://matplotlib.org/3.2.1/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py">チュートリアル</a>にある方法をでファイルをプロット<br>

使用するmatplotlibのメソッド<br>
<a href= "https://matplotlib.org/3.2.1/api/image_api.html#matplotlib.image.imread" >matplotlib.image.imread()</a><br>
<a href= "https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow" >matplotlib.pyplot.imshow()</a>
    </p>


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

f_name="Parrots.jpg" #jpegファイル名


img = mpimg.imread(f_name) #jpegファイルの読み込み

imgplot = plt.imshow(img)  #jpegファイルの描画


FileNotFoundError: [Errno 2] No such file or directory: 'Parrots.jpg'

<h3>②読み込んだimgの確認</h3>
<p>numpyの<a href="https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray">ndarray</a>クラスの模様、またデータは150×150 pixel のRGB3色データ(画素は0～256の整数型(unit8))の模様</p>

In [ ]:
type(img)

In [ ]:
img.shape

In [ ]:
img.dtype

In [ ]:
img.size,150*150*3

<h3>③imgplotの確認</h3>

In [ ]:
type(imgplot)

In [ ]:
dir(imgplot)

In [ ]:
type(imgplot.axes)

In [ ]:
dir(imgplot.axes)

<p>調べた結果
imgplotは<a href="https://matplotlib.org/3.2.1/api/image_api.html#matplotlib.image.AxesImage">matplotlib.image.AxesImage</a>クラスでそのアトリビュートにaxesがあり、

その型を調べると<a>matplotlib.axes._subplots.AxesSubplot</a>クラスでアトリビュートを見る限りおそらく<a href="https://matplotlib.org/3.2.1/api/axes_api.html#the-axes-class">matplotlib.axes</a>の近縁のようでaxexクラスのメソッドが有効そうなのでとりあえずset_xtick(),set_ytick()メソッドで目盛を外してみる。(引数としてからのリスト<a>[]</a>を渡す)</p>

In [ ]:
imgplot = plt.imshow(img)
imgplot.axes.set_xticks([]) #x軸の目盛を削除
imgplot.axes.set_yticks([]) #y軸の目盛を削除

<h3>④グレイスケールとRGBの分離とマルチプロット</h3>

<p>まず画素のRGB各RGBのみを取り出す。
<a href= "https://matplotlib.org/3.2.1/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py">チュートリアル</a>のやり方を参考にすると各色一行のコードで記述可能。(おをらくnumpyの機能)</p>

<p>チュートリアル曰く画素がRGB(またはRGBA)の場合0-255整数(uint8)か0-1の少数(float32)をサポートするが、グレイスケールの場合float32型の値のみのサポートとのこなのでとりあえず[0:1]にスケーリング</p>

<p>※チュートリアルではグレースケールと言っているが厳密には単色画素もしくは<a href= "https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow">imshow()</a>のリファランスの言葉を借りてscalar dataというのが妥当そう。</p>

<p>RGBの画素の平均値をとってgscale_imgとする。これはnumpy.ndarayの<a href="https://numpy.org/doc/stable/reference/generated/numpy.ndarray.mean.html#numpy.ndarray.mean">mean()</a>メソッドで簡単にできる。
ついでに[0:1]にスケーリング。
また後述の理由により反転</p>

In [ ]:
red_img   = img[:,:,0] /255.  #R値の取り出し及び[0:256](uint8) →[0:1](float)のスケーリング
green_img = img[:,:,1] /255.  #G値の取り出し及び[0:256](uint8) →[0:1](float)のスケーリング
blue_img  = img[:,:,2] /255.  #B値の取り出し及び[0:256](uint8) →[0:1](float)のスケーリング

gscale_img = 1. - img.mean(2)/255. #RGBの平均値取得[0:256](uint8) →[0:1](float)のスケーリング、反転

<p>マルチプロットするための<a href ="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.subplots.html?highlight=subplots#matplotlib.pyplot.subplots">matlibplo.pyplot.subplots()</a>で生成されたAxesオブジェクト(以下ではaxs[][])には<a href="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow">1show()</a>メソッドがあり、ファランスを見る限り<a href= "https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow" >matplotlib.pyplot.imshow()</a>と内容は同じそうなのでaxs[i][j].imshow()で作った各データのマルチプロットでの描画を試してみる。</p>

In [ ]:
fig, axs = plt.subplots(2, 3,figsize=[10.5,7.])   #2x3のマルチっプロットの作成


In [ ]:
type(axs[0][0])

In [ ]:
dir(axs[0][0])

<p>ここで<a href="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow">いｍshow()</a>の引数cmapは単色画像(scalor data)を表示する場合のカラーマップを指定するもので、この<a href="https://matplotlib.org/3.2.1/tutorials/colors/colormaps.html">リンク</a>先が詳しい</p>

<p>
RGB各職に対応したグラデーションパターン"Reds","Greens","Blues"もあり各色の表示にはそれを使う、ただし、グレースケールも"Greys"としてあるが先ほどRGBの平均値を使う場合は注意が必要、RGBは光の三原色なので平均値をとると輝度値(0が黒で1が白)になる、一方"Greys"は値が大きくなるにつれて白→黒となるパターンとなる。これが先ほどグレースケールで反転を使った理由。
</p>

In [ ]:
fig, axs = plt.subplots(2, 3,figsize=[10.5,7.])   #2x3のマルチっプロットの作成


axs[0][0].imshow(img)
axs[0][0].set_xticks([])
axs[0][0].set_yticks([])
axs[0][0].set_title("Original Image")

axs[0][1].imshow(gscale_img,cmap="Greys")
axs[0][1].set_xticks([])
axs[0][1].set_yticks([])
axs[0][1].set_title("Grey")

axs[0][2].imshow(1.-gscale_img,cmap="Greys")    #グレースケールの反転
axs[0][2].set_xticks([])
axs[0][2].set_yticks([])
axs[0][2].set_title("Negative Grey")

axs[1][0].imshow(red_img,cmap="Reds")
axs[1][0].set_xticks([])
axs[1][0].set_yticks([])
axs[1][0].set_title("Grey")

axs[1][1].imshow(green_img,cmap="Greens")
axs[1][1].set_xticks([])
axs[1][1].set_yticks([])
axs[1][1].set_title("Green")

axs[1][2].imshow(blue_img,cmap="Blues")
axs[1][2].set_xticks([])
axs[1][2].set_yticks([])
axs[1][2].set_title("Blue")


<p>作った単色画質にたいして色々演算などをやっていても思い通りに描画できないので調べてみると、<a href="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow">imshow()</a>メソッドでは単色スケールの表示はデフォルトでは画素値の最大値=>1、最小値=>0となるようにオートスケールさおり、そのスケールを固定するためにはimshow()のNormパラメータで設定可能とのこと。</p>
<p>Normパラメータは<a href="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.colors.Normalize.html#matplotlib.colors.Normalize"></a>matplotlib.colors.Normalizeクラスを渡す必要があり、matplotlib.colorsモジュールをmcolとしてインポートしてnormalize_fixというインスタンス(0～1に固定の意味、、)を作ってみた。
以上によりオートスケールの無効化を試してみる。
(おそらくカラーバー表示のグラフとかでも同様の手法が応用できると思われる。)</p>

<p>出力の下段の通り画素値を半分にしたときオートスケール有効だと元の画像と変わらないがオートスケールを無効にすると薄く表示される。</p>


In [ ]:
import matplotlib.colors as mpcol

normalize_fix=mpcol.Normalize(0,1)

fig, axs = plt.subplots(2, 3,figsize=[10.5,7.]) 

axs[0][0].imshow(img)
axs[0][0].set_xticks([])
axs[0][0].set_yticks([])
axs[0][0].set_title("Original Image")

axs[0][1].imshow(gscale_img,cmap="Greys",norm=normalize_fix)
axs[0][1].set_xticks([])
axs[0][1].set_yticks([])
axs[0][1].set_title("Grey without Autoscale")

axs[0][2].imshow(1.-gscale_img,cmap="Greys",norm=normalize_fix)
axs[0][2].set_xticks([])
axs[0][2].set_yticks([])
axs[0][2].set_title("Negative Grey without Autoscale")

axs[1][0].imshow(gscale_img,cmap="Greys")
axs[1][0].set_xticks([])
axs[1][0].set_yticks([])
axs[1][0].set_title("Grey with Autoscale")

axs[1][1].imshow(gscale_img/2.,cmap="Greys")
axs[1][1].set_xticks([])
axs[1][1].set_yticks([])
axs[1][1].set_title("Grey/2.0 with Autoscale")

axs[1][2].imshow(gscale_img/2.,cmap="Greys",norm=normalize_fix)
axs[1][2].set_xticks([])
axs[1][2].set_yticks([])
axs[1][2].set_title("Grey/2.0 without Autoscale")


<p><a href="https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow">imshow()</a>の説明をよく読むと直接引数としてvmin,vmaxを指定できそうなのでその方法も試してみる。こっちのほうが簡単</p>

In [ ]:

fig, axs = plt.subplots(2, 3,figsize=[10.5,7.]) 

axs[0][0].imshow(img)
axs[0][0].set_xticks([])
axs[0][0].set_yticks([])
axs[0][0].set_title("Original Image")

axs[0][1].imshow(gscale_img,cmap="Greys",vmin=0,vmax=1)
axs[0][1].set_xticks([])
axs[0][1].set_yticks([])
axs[0][1].set_title("Grey without Autoscale")

axs[0][2].imshow(1.-gscale_img,cmap="Greys",vmin=0,vmax=1)
axs[0][2].set_xticks([])
axs[0][2].set_yticks([])
axs[0][2].set_title("Negative Grey without Autoscale")

axs[1][0].imshow(gscale_img,cmap="Greys")
axs[1][0].set_xticks([])
axs[1][0].set_yticks([])
axs[1][0].set_title("Grey with Autoscale")

axs[1][1].imshow(gscale_img/2.,cmap="Greys")
axs[1][1].set_xticks([])
axs[1][1].set_yticks([])
axs[1][1].set_title("Grey/2.0 with Autoscale")

axs[1][2].imshow(gscale_img/2.,cmap="Greys",vmin=0,vmax=1)
axs[1][2].set_xticks([])
axs[1][2].set_yticks([])
axs[1][2].set_title("Grey/2.0 without Autoscale")


調子に乗ってvmin,vmaxをいろいろいじってみる
当然vmin<vmaxでないとエラーが出る。

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=[7.,7.]) 

axs[0][0].imshow(img)
axs[0][0].set_xticks([])
axs[0][0].set_yticks([])
axs[0][0].set_title("Original Image")

axs[0][1].imshow(gscale_img,cmap="Greys",vmin=0,vmax=1)
axs[0][1].set_xticks([])
axs[0][1].set_yticks([])
axs[0][1].set_title("Grey vmin=0,vmin=1")

axs[1][0].imshow(gscale_img,cmap="Greys",vmin=0.5,vmax=1)
axs[1][0].set_xticks([])
axs[1][0].set_yticks([])
axs[1][0].set_title("Grey vmin=0.5,vmax=1")

axs[1][1].imshow(gscale_img,cmap="Greys",vmin=0.0,vmax=0.5)
axs[1][1].set_xticks([])
axs[1][1].set_yticks([])
axs[1][1].set_title("Grey  vmin=0,vmax=0.5")